In [57]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io as skio
from skimage.transform import resize
import pandas as pd
import os
import requests
from pyarrow import feather
import io
import concurrent.futures

import warnings
warnings.filterwarnings("ignore")

In [65]:
from sklearn.preprocessing import MinMaxScaler

def convert_audio_to_mel_image(filepath_to_audio, filepath_to_save, image_size=(128,512), n_mels=128, fmax=8000,):
    signal, sr = librosa.load(filepath_to_audio)
    mels = librosa.power_to_db(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels, fmax=fmax), ref=np.max)
    scaler = MinMaxScaler()
    image = (scaler.fit_transform(mels)*255)
    image = np.flip(image, axis=0)
    #image = 255-image
    resize_image = resize(image, image_size).astype(np.uint8)
    skio.imsave(filepath_to_save, resize_image)

In [66]:
tracks_df = feather.read_feather('data/all_tracks.feather')

In [67]:
tracks_df = tracks_df.dropna(subset=['track_preview_link'])

In [68]:
tracks_df = tracks_df.reset_index(drop=True)
tracks_df

,track_id,track_name,track_preview_link,track_popularity,track_uri,artist_name,artist_id,artist_genres,artist_popularity,release_date
0,79620P7nP5cSS7f8ULHfo8,Ha-le-lu-ja,https://p.scdn.co/mp3-preview/54111ab335474758...,18,spotify:track:79620P7nP5cSS7f8ULHfo8,Klostertaler,6adGYZ9CU5LPwqLYt6yQc7,"[classic schlager, schlager, volksmusik, zille...",36,2007-11-22
1,5k8nNRu2aOjssNyCPzmax3,Gott schütze die Frauen - Radio Version,https://p.scdn.co/mp3-preview/e5bdcf4b6163a4b0...,34,spotify:track:5k8nNRu2aOjssNyCPzmax3,Klostertaler,6adGYZ9CU5LPwqLYt6yQc7,"[classic schlager, schlager, volksmusik, zille...",36,2004-03-09
2,7Jj23tl4vV4SblQgYZgCAL,Her mit meinen Hennen (Der Goggala),https://p.scdn.co/mp3-preview/56e52c31b64dad3d...,31,spotify:track:7Jj23tl4vV4SblQgYZgCAL,Klostertaler,6adGYZ9CU5LPwqLYt6yQc7,"[classic schlager, schlager, volksmusik, zille...",36,2007-11-22
3,7bhIlje38pzEuPllpto7JK,Pico-Bello - Radio Version,https://p.scdn.co/mp3-preview/542685c57d976351...,29,spotify:track:7bhIlje38pzEuPllpto7JK,Klostertaler,6adGYZ9CU5LPwqLYt6yQc7,"[classic schlager, schlager, volksmusik, zille...",36,2004-03-09
4,6cvUVbY2L8Ky6kYGxGrEgQ,Mir Geh´n No Lang Net Ins Bett - Radio Version,https://p.scdn.co/mp3-preview/c7e99e589e7413b7...,28,spotify:track:6cvUVbY2L8Ky6kYGxGrEgQ,Klostertaler,6adGYZ9CU5LPwqLYt6yQc7,"[classic schlager, schlager, volksmusik, zille...",36,2004-03-09
...,...,...,...,...,...,...,...,...,...,...
563288,7310HkEbJ2SLziIte7WZSi,Gott is a Dirndl,https://p.scdn.co/mp3-preview/92807dcc461ecf74...,43,spotify:track:7310HkEbJ2SLziIte7WZSi,Melissa Naschenweng,00unybLgvbIqKxx0OTyYYZ,"[partyschlager, schlager]",47,None
563289,1IzYuIgrzZhUp9yomTowk0,Die ganze Nacht - Dance Mix,https://p.scdn.co/mp3-preview/81b696e709b644cf...,41,spotify:track:1IzYuIgrzZhUp9yomTowk0,Melissa Naschenweng,00unybLgvbIqKxx0OTyYYZ,"[partyschlager, schlager]",47,None
563290,2sUXc7xn5Xy3mmTcLfOxHo,Braungebrannte Haut,https://p.scdn.co/mp3-preview/ed41988000214477...,40,spotify:track:2sUXc7xn5Xy3mmTcLfOxHo,Melissa Naschenweng,00unybLgvbIqKxx0OTyYYZ,"[partyschlager, schlager]",47,None
563291,0OxNysTH08lf6SS2GInHx9,Die Nachbarin,https://p.scdn.co/mp3-preview/765395036326c9b6...,40,spotify:track:0OxNysTH08lf6SS2GInHx9,Melissa Naschenweng,00unybLgvbIqKxx0OTyYYZ,"[partyschlager, schlager]",47,None


In [75]:
def download_preview_with_index(index):
    track_url = tracks_df.loc[index, 'track_preview_link']

    preview = requests.get(track_url)
    
    track_id = tracks_df.loc[index, 'track_id']

    filename = f'data/Spotify/mp3s/{track_id}.mp3'
    png_name = f'data/Spotify/pngs/{track_id}.png'

    with open(filename, 'wb') as f:
        f.write(preview.content)

    convert_audio_to_mel_image(filename, png_name)

    os.remove(filename)
    
    print(index, end='\r')

In [76]:
pd.options.mode.chained_assignment = None

indices = tracks_df.index
MAX_THREADS = 30
threads = min(MAX_THREADS, len(indices))
count = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_preview_with_index, indices)

KeyboardInterrupt: 

In [77]:
last_index = 66143